<a href="https://colab.research.google.com/github/yongjulee0213/ale_python_interface/blob/master/C3_W4_Lab_2_irish_lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
data=open('/content/irish-lyrics-eof.txt').read()
corpus=data.lower().split('\n')
print(corpus)

['come all ye maidens young and fair', 'and you that are blooming in your prime', 'always beware and keep your garden fair', 'let no man steal away your thyme', 'for thyme it is a precious thing', 'and thyme brings all things to my mind', 'nlyme with all its flavours, along with all its joys', 'thyme, brings all things to my mind', 'once i and a bunch of thyme', 'i thought it never would decay', 'then came a lusty sailor', 'who chanced to pass my way', 'and stole my bunch of thyme away', 'the sailor gave to me a rose', 'a rose that never would decay', 'he gave it to me to keep me reminded', 'of when he stole my thyme away', 'sleep, my child, and peace attend thee', 'all through the night', 'guardian angels god will send thee', 'soft the drowsy hours are creeping', 'hill and dale in slumber sleeping', 'i my loving vigil keeping', 'while the moon her watch is keeping', 'while the weary world is sleeping', 'oer thy spirit gently stealing', 'visions of delight revealing', 'breathes a pure 

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words=len(tokenizer.word_index)+1
print(total_words)

2690


In [20]:
input_sequences=[]
for line in corpus:
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_sequences=token_list[:i+1]
    input_sequences.append(n_gram_sequences)

max_sequence_len=max([len(x) for x in input_sequences])
print(max_sequence_len)

16


In [21]:
#패딩
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_sequences=pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre')
print(input_sequences)

[[   0    0    0 ...    0   51   12]
 [   0    0    0 ...   51   12   96]
 [   0    0    0 ...   12   96 1217]
 ...
 [   0    0    0 ...    0   47  105]
 [   0    0    0 ...   47  105  138]
 [   0    0    0 ...  105  138  184]]


In [24]:
import tensorflow as tf
xs=input_sequences[:,:-1]
labels=input_sequences[:,-1]
print(len(labels))


12038


In [23]:
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

print(f'xs: {xs}')
print(f'ys: {ys}')

xs: [[  0   0   0 ...   0   0  51]
 [  0   0   0 ...   0  51  12]
 [  0   0   0 ...  51  12  96]
 ...
 [  0   0   0 ...   0   0  47]
 [  0   0   0 ...   0  47 105]
 [  0   0   0 ...  47 105 138]]
ys: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, Dense,LSTM
from tensorflow.keras.optimizers import Adam

model=Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam=Adam(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])
history=model.fit(xs, ys, epochs=100,verbose=1)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


377/377 [==============================] - 36s 83ms/step - loss: 6.6638 - accuracy: 0.0749
Epoch 2/100
377/377 [==============================] - 42s 111ms/step - loss: 5.7891 - accuracy: 0.1083
Epoch 3/100
377/377 [==============================] - 39s 103ms/step - loss: 4.9781 - accuracy: 0.1552
Epoch 4/100
377/377 [==============================] - 28s 75ms/step - loss: 4.0638 - accuracy: 0.2244
Epoch 5/100
377/377 [==============================] - 26s 70ms/step - loss: 3.2182 - accuracy: 0.3283
Epoch 6/100
377/377 [==============================] - 36s 95ms/step - loss: 2.5136 - accuracy: 0.4384
Epoch 7/100
377/377 [==============================] - 38s 101ms/step - loss: 2.0112 - accuracy: 0.5298
Epoch 8/100
377/377 [==============================] - 31s 83ms/step - loss: 1.6455 - accuracy: 0.6080
Epoch 9/100
377/377 [==============================] - 26s 69ms/step - loss: 1.3808 - accuracy: 0.6697
Epoch 10/100
377/377 [==============================] - 25s 67ms/step - loss: 1.20

In [ ]:
import matplotlib.pyplot as plt
def plot_graph(history,string):
  plt.plot(history.history[string])
  plt.xlabel('epochs')
  plt.ylabel(string)
  plt.show()

plot_graph(history,'accuracy')

In [ ]:
from prompt_toolkit import output
import numpy as np

seed_text='help me obi-wan kinobi youre my only hope'

next_words=100

for _ in range(next_words):
  token_list=tokenizer.texts_to_sequences([seed_text])[0]
  token_list=pad_sequences(token_list, maxlen=max_sequence_len,padding='pre')
  probabilities=model.predict(token_list)
  predicted=np.argmax(probabilities,axis=-1)[0]
  if predicted!=0:
    output_word=tokenizer.index_word[predicted]
    seed_text+=" "+ output_word
print(seed_text)